In [1]:
# model의 setting을 저장한 yaml 파일 확인
import os
os.listdir('../experiments/cityscapes')

['.ipynb_checkpoints',
 'seg_hrnet_w48_trainval_ohem_512x1024_sgd_lr1e-2_wd5e-4_bs_12_epoch484x2.yaml',
 'seg_hrnet_ocr_w48_train_512x1024_sgd_lr1e-2_wd5e-4_bs_16_epoch484_paddle.yaml',
 'seg_hrnet_ocr_w48_trainval_512x1024_sgd_lr1e-2_wd5e-4_bs_12_epoch484.yaml',
 'seg_hrnet_w48_train_512x1024_sgd_lr1e-2_wd5e-4_bs_12_epoch484_paddle.yaml',
 'seg_hrnet_w48_train_ohem_512x1024_sgd_lr1e-2_wd5e-4_bs_12_epoch484.yaml',
 'seg_hrnet_w48_train_512x1024_sgd_lr1e-2_wd5e-4_bs_12_epoch484.yaml',
 'seg_hrnet_ocr_w48_train_512x1024_sgd_lr1e-2_wd5e-4_bs_16_epoch484_paddle_custom.yaml',
 'seg_hrnet_w48_train_512x1024_sgd_lr1e-2_wd5e-4_bs_16_epoch484_paddle.yaml',
 'seg_hrnet_ocr_w48_train_512x1024_sgd_lr1e-2_wd5e-4_bs_12_epoch484.yaml',
 'seg_hrnet_w48_trainval_512x1024_sgd_lr1e-2_wd5e-4_bs_12_epoch484x2.yaml']

In [2]:
# 모델 load test
from models.seg_hrnet_ocr import get_seg_model
get_seg_model()

# config 파일을 필요로함 

TypeError: get_seg_model() missing 1 required positional argument: 'cfg'

In [3]:
# form import 문처럼 config를 사용이 가능함  
# 실제로 config load를 하면 model(seg HRNet)과 config의 내용이 맞지 않음 
from config import config
get_seg_model(config)

KeyError: 'STAGE1'

In [83]:
# cofing 파일을 보면 기존의 config을 생성후에 이를 파일에서 읽어서 하는 구조임
# updata_conifg 함수를 수정해서 사용하도록 만듬

import yaml
from config import config
from yacs.config import CfgNode as CN
from models.seg_hrnet_ocr import get_seg_model


def update_config(cfg, cfg_file_path):
    cfg.defrost()

    # 파일에서 YAML 설정 읽기
    with open(cfg_file_path, 'r') as f:
        yaml_cfg = yaml.safe_load(f)
    
    yaml_cfg['MODEL']['PRETRAINED'] = '../model_ckpt/hrnet_ocr_cs_trainval_8227_torch11.pth'
    #yaml_cfg['MODEL']['NUM_OUTPUTS'] = 3
    cfg.merge_from_other_cfg(CN(yaml_cfg))
    cfg.freeze()

In [84]:
import os
import torch
import numpy as np
import segmentation_models_pytorch as smp
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader


class SkinDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.image_dir = os.path.join(root_dir, 'images')
        self.mask_dir = os.path.join(root_dir, 'masks')
        
        # Add a filter to exclude directories
        self.image_files = sorted([f for f in os.listdir(self.image_dir) if os.path.isfile(os.path.join(self.image_dir, f))])
        self.mask_files = sorted([f for f in os.listdir(self.mask_dir) if os.path.isfile(os.path.join(self.mask_dir, f))])
        
        self.transform = transform
        
    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_files[idx])
        mask_path = os.path.join(self.mask_dir, self.mask_files[idx])

        image = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")

        unique_values = np.unique(mask)
        image = self.transform(image)

        mask_np = np.array(mask)
        masks = [(mask_np == val).astype(int) for val in range(3)]
        mask = np.stack(masks, axis=0)
        mask = torch.tensor(mask, dtype=torch.float32)
        return image, mask


In [85]:
batch_size = 2
transform = transforms.Compose([
    transforms.Resize((640, 640)),
    transforms.ToTensor()
])
train_data = SkinDataset(root_dir='../../../../../Datasets/NewSkinDataset/original/train', transform=transform)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

test_data = SkinDataset(root_dir='../../../../../Datasets/NewSkinDataset/original/val', transform=transform)
test_loader = DataLoader(test_data, batch_size=10, shuffle=True)


In [88]:
from models.seg_hrnet_ocr import get_seg_model
update_config(config, '../experiments/cityscapes/seg_hrnet_ocr_w48_train_512x1024_sgd_lr1e-2_wd5e-4_bs_16_epoch484_paddle.yaml')
model = get_seg_model(config)

set()
{'loss.criterion1.weight', 'loss.criterion0.weight'}


In [87]:
for batch in test_loader:
    image , mask = batch
    result = model(image)
    print(len(result) , type(result) , result[0].shape)
    break

2 <class 'list'> torch.Size([10, 19, 160, 160])


In [80]:
# config 파일에서 num_class를 변경하면 다음과 같은 에러가 뜸
update_config(config, '../experiments/cityscapes/seg_hrnet_ocr_w48_train_512x1024_sgd_lr1e-2_wd5e-4_bs_16_epoch484_paddle_custom.yaml')
model = get_seg_model(config)

set()
{'loss.criterion1.weight', 'loss.criterion0.weight'}


In [15]:
update_config(config, '../experiments/cityscapes/seg_hrnet_ocr_w48_train_512x1024_sgd_lr1e-2_wd5e-4_bs_16_epoch484_paddle.yaml')
model = get_seg_model(config)
model


set()
{'loss.criterion1.weight', 'loss.criterion0.weight'}


HighResolutionNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): SyncBatchNorm(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn2): SyncBatchNorm(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): SyncBatchNorm(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): SyncBatchNorm(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): SyncBatchNorm(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): 

In [72]:
# torchsummary는 에러남 
from torchsummary import summary

update_config(config, '../experiments/cityscapes/seg_hrnet_ocr_w48_train_512x1024_sgd_lr1e-2_wd5e-4_bs_16_epoch484_paddle.yaml')
model = get_seg_model(config)

if torch.cuda.is_available():
    model = model.cuda()

input_size = (3, 640, 640)
summary(model, input_size)


set()
{'loss.criterion1.weight', 'loss.criterion0.weight'}


AttributeError: 'list' object has no attribute 'size'

In [74]:
# torchsummaryX로 변경해서 model summary 확인
from torchsummaryX import summary
import pandas as pd

update_config(config, '../experiments/cityscapes/seg_hrnet_ocr_w48_train_512x1024_sgd_lr1e-2_wd5e-4_bs_16_epoch484_paddle.yaml')
model = get_seg_model(config)

# 모델을 CUDA로 이동 (CUDA가 사용 가능한 경우)
if torch.cuda.is_available():
    model = model.cuda()

input_size = (3, 1280, 1280)  # 리스트가 아닌 튜플로 변경
summary_df = summary(model, torch.zeros((1, *input_size)).cuda())  # 모델에 전달할 텐서를 직접 생성합니다.

# 데이터프레임을 CSV 파일로 저장
summary_df.to_csv('model_summary.csv')


set()
{'loss.criterion1.weight', 'loss.criterion0.weight'}
                                                         Kernel Shape  \
Layer                                                                   
0_conv1                                                 [3, 64, 3, 3]   
1_bn1                                                            [64]   
2_relu                                                              -   
3_conv2                                                [64, 64, 3, 3]   
4_bn2                                                            [64]   
...                                                               ...   
904_ocr_distri_head.conv_bn_dropout.Conv2d_0        [1024, 512, 1, 1]   
905_ocr_distri_head.conv_bn_dropout.1.SyncBatch...              [512]   
906_ocr_distri_head.conv_bn_dropout.1.ReLU_1                        -   
907_ocr_distri_head.conv_bn_dropout.Dropout2d_2                     -   
908_cls_head                                          [512, 19, 1

/home/nplab/.local/lib/python3.8/site-packages/torchsummaryX/torchsummaryX.py:101: FutureWarning: The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_sum = df.sum()
